In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Pré-Processamento
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_validate

# Classificadores
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Avaliação
from sklearn.metrics import accuracy_score, precision_score, recall_score,  roc_curve, auc

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("heart.csv")
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,caa,output
0,63,1,3,145,233,1,0,150,0,0,1
1,37,1,2,130,250,0,1,187,0,0,1
2,41,0,1,130,204,0,0,172,0,0,1
3,56,1,1,120,236,0,1,178,0,0,1
4,57,0,0,120,354,0,1,163,1,0,1


In [3]:
# Separando as vaiáveis em categóricas e numéricas 
cat_cols = ['sex', 'cp', 'fbs', 'restecg', 'exng', 'caa']
num_cols = ['age','trtbps', 'chol', 'thalachh']

### Separando os conjuntos para a modelagem

- Base de Treino: Conjunto de características e conjunto de resposta. São os dados com os quais o modelo vai aprender
- Base de Teste: Conjunto de dados em que vamos testar se o modelo aprendeu bem

In [4]:
X = df.drop('output', axis = 1)
y = df['output']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)
X_train.shape, X_test.shape

((242, 10), (61, 10))

### Pré-processamento

**Normalização**

In [6]:
scaler = StandardScaler()
X_train.loc[:,num_cols] = scaler.fit_transform(X_train.loc[:, num_cols])
X_test.loc[:,num_cols] = scaler.transform(X_test.loc[:, num_cols])

**Enconding das variáveis Categóricas**

In [7]:
#como nesta base as variáveis categóricas já são um número vamos apenas exemplificar o uso do encoder
exemplo = np.where(X_train['sex'] == 1, "Fem", "Masc")
exemplo

array(['Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc', 'Fem', 'Fem',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc', 'Masc',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc', 'Fem',
       'Masc', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem',
       'Masc', 'Masc', 'Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc',
       'Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc',
       'Fem', 'Masc', 'Masc', 'Masc', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem',
       'Fem', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem',
       'Fem', 'Fem', 'Masc', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc', 'Fem',
       'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc', 'Fem',
       'Fem', 'Masc', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Masc', 'Fem',
       'Fem', 'Fem', 'Masc', 'Fem', 'Masc', 'Masc', 'Fem', 'Masc', 'Fem',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', '

In [8]:
le = preprocessing.LabelEncoder()
le.fit(exemplo)
le.classes_ # quantas classes únicas foram encontradas

array(['Fem', 'Masc'], dtype='<U4')

In [9]:
# Encoding
exemplo_cod = le.transform(exemplo)
exemplo_cod

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1],
      dtype=int64)

In [10]:
# Processo reverso
le.inverse_transform(exemplo_cod)

array(['Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc', 'Fem', 'Fem',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc', 'Masc',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc', 'Fem',
       'Masc', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem',
       'Masc', 'Masc', 'Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc',
       'Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc',
       'Fem', 'Masc', 'Masc', 'Masc', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem',
       'Fem', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem',
       'Fem', 'Fem', 'Masc', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc', 'Fem',
       'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Fem', 'Masc', 'Masc', 'Fem',
       'Fem', 'Masc', 'Masc', 'Fem', 'Masc', 'Fem', 'Fem', 'Masc', 'Fem',
       'Fem', 'Fem', 'Masc', 'Fem', 'Masc', 'Masc', 'Fem', 'Masc', 'Fem',
       'Fem', 'Fem', 'Fem', 'Fem', 'Fem', 'Fem', '

### Métodos de avaliação

**Matriz de confusão**

| - | SIM (predito) | NÃO (predito) |
| --------  | ------------------- | --------------------- |
| **SIM (real)**  | Verdadeiro Positivo (VP) | False Negativo (FN)| 
| **NÃO (real)**  | Falso Positivo (FP) | Verdadeiro Negativo (VN) | 

- Verdadeiros Positivos: classificação correta da classe Positivo
- Falsos Negativos: erro em que o modelo previu a classe Negativo quando o valor real era classe Positivo
- Falsos Positivos: erro em que o modelo previu a classe Positivo quando o valor real era classe Negativo
- Verdadeiros Negativos: classificação correta da classe Negativo

**Métricas de Avaliação**
- Acurácia: indica uma performance geral do modelo. Dentre todas as classificações, quantas o modelo classificou corretamente (VP + VN)/(VP + VN + FP + FN)
- Precisão: dentre todas as classificações de classe Positivo que o modelo fez, quantas estão corretas VP/(VP+FP)
- Recall/Revocação/Sensibilidade: dentre todas as situações de classe Positivo como valor esperado, quantas estão corretas VP/(VP+FN)

### Modelagem

1. Escolha o seu classificador e busque na documentação como usá-lo
2. A função *fit* se refere a etapa de aprendizado
3. A função *predict* irá utilizar o aprendizado adquirido anteriormente para classificar um novo conjunto de dados

**Para quem quiser mais**:
- https://medium.com/fintechexplained/what-is-grid-search-c01fe886ef0a
- https://www.kaggle.com/willkoehrsen/intro-to-model-tuning-grid-and-random-search

**Regressão Logística**

In [24]:
log_model = LogisticRegression(solver='liblinear')
log_model.fit(X, y)

LogisticRegression(solver='liblinear')

In [25]:
print(accuracy_score(y, log_model.predict(X)))
print(precision_score(y, log_model.predict(X)))
print(recall_score(y, log_model.predict(X)))

0.8118811881188119
0.8033707865168539
0.8666666666666667


**Árvores de Decisão**

In [13]:
decision_model = DecisionTreeClassifier(max_depth=4)
decision_model.fit(X , y)

DecisionTreeClassifier(max_depth=4)

In [14]:
print(accuracy_score(y, decision_model.predict(X)))
print(precision_score(y, decision_model.predict(X)))
print(recall_score(y, decision_model.predict(X)))

0.8811881188118812
0.8486486486486486
0.9515151515151515


**Random Forest**

In [20]:
rf = RandomForestClassifier(criterion='entropy', max_depth=2)
rf.fit(X, y)

RandomForestClassifier(criterion='entropy', max_depth=2)

In [21]:
print(accuracy_score(y, rf.predict(X)))
print(precision_score(y, rf.predict(X)))
print(recall_score(y, rf.predict(X)))

0.8217821782178217
0.8032786885245902
0.8909090909090909


### Validação Cruzada

<div>
<center><img src="kfold.jpg" width="600"/></center>
</div>

In [ ]:
k = 4
#kf = KFold(n_splits=k, shuffle=True)
kf = KFold(n_splits=k, shuffle=False)
model = LogisticRegression(solver='liblinear')
#model =  DecisionTreeClassifier(max_depth=4)

acc_score = []
 
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))